In [1]:
import os
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import Dict, List, Tuple, NamedTuple, Callable
import scml
import mylib

In [2]:
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()

In [3]:
%%time
df = pd.read_csv("input/validation_data.csv", engine="c", low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30108 entries, 0 to 30107
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   worker      30108 non-null  int64 
 1   less_toxic  30108 non-null  object
 2   more_toxic  30108 non-null  object
dtypes: int64(1), object(2)
memory usage: 705.8+ KB
Wall time: 225 ms


In [4]:
texts = set(df["less_toxic"]) | set(df["more_toxic"])
texts = list(texts)
df = pd.DataFrame(data={"text": texts})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14251 entries, 0 to 14250
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    14251 non-null  object
dtypes: object(1)
memory usage: 111.5+ KB


# Preprocess Text
Speed dropped from 1200 to 1000 it/s

In [5]:
def pre1(row) -> str:
    return mylib.pre1(row["text"])


def pre2(row) -> str:
    return mylib.pre2(row["text1"])


def pre3(row) -> str:
    return mylib.pre3(row["text2"])


col = "text1"
print(col)
df[col] = df.progress_apply(pre1, axis=1)
col = "text2"
print(col)
df[col] = df.progress_apply(pre2, axis=1)
col = "text3"
print(col)
df[col] = df.progress_apply(pre3, axis=1)

text1


100%|████████████████████████████████████████| 14251/14251 [00:50<00:00, 283.77it/s]


text2


100%|█████████████████████████████████████████| 14251/14251 [14:01<00:00, 16.94it/s]


text3


100%|█████████████████████████████████████████| 14251/14251 [03:15<00:00, 72.81it/s]


In [6]:
cols = ["text", "text1", "text2", "text3"]
df[cols].sample(10).head(10)

,text,text1,text2,text3
8746,Jeff G. ツ likes to rape goats while illegal aliens choke him and shove broken glass in his ass.,Jeff G. likes to rape goats while illegal aliens choke him and shove broken glass in his ass.,Jeff G. likes to rape goats while illegal aliens choke him and shove broken glass in his ass.,jeff g like to rape goat while illegal alien choke he and shove broken glass in his ass
5149,"""\n\n Your edit to C. \n\n Please stop. If you continue to vandalize Wikipedia, you will be blocked from editing. · Talk """,""" Your edit to C. Please stop. If you continue to vandalize Wikipedia, you will be blocked from editing. Talk """,""" Your edit to C. Please stop. If you continue to vandalize Wikipedia, you will be blocked from editing. Talk """,your edit to c please stop if you continue to vandalize wikipedia you will be block from edit talk
10696,"I wish it were that simple Sidd. I tried appealing to wafulz who seemed to not understand or care about what was going on. I think for the most part he is the overseer of the page. I am sorry if I barked at you. I have been blindsided by ignorant people here from the beginning. I am interested in the 'truth' of this as you seem to be. You are probably right about getting someones attention about Technocrate. He did these same exact reverts when I first arrived then disappeared for a while. He has never creatively done any thing.\n Would you mind doing it, blowing the whistle, or bringing it to the attention of wafulz, or better yet a different editor admin. I would appreciate that. He does not seem disposed toward me. I have tried many times to engage him.","I wish it were that simple Sidd. I tried appealing to wafulz who seemed to not understand or care about what was going on. I think for the most part he is the overseer of the page. I am sorry if I barked at you. I have been blindsided by ignorant people here from the beginning. I am interested in the 'truth' of this as you seem to be. You are probably right about getting someones attention about Technocrate. He did these same exact reverts when I first arrived then disappeared for a while. He has never creatively done any thing. Would you mind doing it, blowing the whistle, or bringing it to the attention of wafulz, or better yet a different editor admin. I would appreciate that. He does not seem disposed toward me. I have tried many times to engage him.","I wish it were that simple Sidd. I tried appealing to wafulz who seemed to not understand or care about what was going on. I think for the most part he is the overseer of the page. I am sorry if I barked at you. I have been blindsided by ignorant people here from the beginning. I am interested in the 'truth' of this as you seem to be. You are probably right about getting someones attention about Technocrate. He did these same exact reverts when I first arrived then disappeared for a while. He has never creatively done any thing. Would you mind doing it, blowing the whistle, or bring it to the attention of wafulz, or better yet a different editor admin. I would appreciate that. He does not seem disposed toward me. I have tried many times to engage him.",i wish it be that simple sidd i try appeal to wafulz who seem to not understand or care about what be go on i think for the most part he be the overseer of the page i be sorry if i bark at you i have be blindside by ignorant people here from the beginning i be interested in the truth of this as you seem to be you be probably right about get someone attention about technocrate he do these same exact revert when i first arrive then disappear for a while he have never creatively do any thing would you mind do it blow the whistle or bring it to the attention of wafulz or well yet a different editor admin i would appreciate that he do not seem dispose toward i i have try many time to engage he
6412,"I am very surprised and have to say that I think such action is both unnecessary and is being ridiculous. Having just read the

In [7]:
df[cols].sample(10).head(10)

,text,text1,text2,text3
1316,poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. \n\npoop. \npoop. \npoop.,poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poop. poo

In [8]:
df[cols].sample(10).head(10)

,text,text1,text2,text3
11886,"Cathel, one thing you ignore and One Night In Hackney has never been able to explain is why Bobby Sands and other Irish Republicans are the only people with pages on Wikipedia where this rule applies to. They are the only pages where factual informaiton is being left out. Cathel, you cannot deny that One Night In Hacknkey is targeting Irish Republican Catholics. You don't see him removing Jewish categories from any other pages, and there are many people in Jewish categories who are not notable for being Jewish. So then why does this rule only apply to Irish Republican Catholics? And of course, the church will condemn any form of violence. Mainstream Muslims leaders also condemn terrorism, does not mean Islamic terrorists should not be in Muslim categories? By your logic, yes. So if I start removing terrorists from Muslim categories, I can count on you to back me up when a person readds the categories?","Cathel, one thing you ignore and One Night In Hackney has never been able to explain is why Bobby Sands and other Irish Republicans are the only people with pages on Wikipedia where this rule applies to. They are the only pages where factual informaiton is being left out. Cathel, you cannot deny that One Night In Hacknkey is targeting Irish Republican Catholics. You don't see him removing Jewish categories from any other pages, and there are many people in Jewish categories who are not notable for being Jewish. So then why does this rule only apply to Irish Republican Catholics? And of course, the church will condemn any form of violence. Mainstream Muslims leaders also condemn terrorism, does not mean Islamic terrorists should not be in Muslim categories? By your logic, yes. So if I start removing terrorists from Muslim categories, I can count on you to back me up when a person readds the categories?","Cathel, one thing you ignore and One Night In Hackney has never been able to explain is why Bobby Sands and other Irish Republicans are the only people with pages on Wikipedia where this rule applies to. They are the only pages where factual informaiton is being left out. Cathel, you cannot deny that One Night In Hacknkey is targeting Irish Republican Catholics. You do not see him removing Jewish categories from any other pages, and there are many people in Jewish categories who are notable for being Jewish. So then why does this rule only apply to Irish Republican Catholics? And of course, the church will condemn any form of violence. Mainstream Muslims leaders also condemn terrorism, does not mean Islamic terrorists should not be in Muslim categories? By your logic, yes. So if I start removing terrorists from Muslim categories, I can count on you to back me up when a person readds the categories?",cathel one thing you ignore and one night in hackney have never be able to explain be why bobby sands and other irish republicans be the only people with page on wikipedia where this rule apply to they be the only page where factual informaiton be be leave out cathel you can not deny that one night in hacknkey be target irish republican catholics you do not see he remove jewish category from any other page and there be many people in jewish category who be notable for be jewish so then why do this rule only apply to irish republican catholics and of course the church will condemn any form of violence mainstream muslims leader also condemn terrorism do not mean islamic terrorist should not be in muslim category by your logic yes so if i start remove terrorist from muslim category i can count on you to back i up when a person readd the category
12459,Making Everbody's Life a Misery \n\nI see that I am not the first person that you have deliberately targetted in a smear campaign. You are a vandal and a disgrace to the good name of Wikpedia.\n\nShame on you.,Making Everbody's Life a Misery I see that I am not the first person that you have deliberately targetted in a smear campaign. You are a vandal and a d

# Review data

In [9]:
df[cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14251 entries, 0 to 14250
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    14251 non-null  object
 1   text1   14251 non-null  object
 2   text2   14251 non-null  object
 3   text3   14251 non-null  object
dtypes: object(4)
memory usage: 445.5+ KB


In [10]:
%%time
df[cols].to_parquet("output/pre_val.parquet", index=False)

Wall time: 130 ms
